In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder, boxscoreadvancedv3

init_notebook_mode(connected=True)

# Get All teams
nba_teams = teams.get_teams()

# Get games of current season
games_per_team = []
for team in nba_teams:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team['id'])
    games = gamefinder.get_data_frames()[0]
    games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
    # Filter games for the 2023-2024 season (without pre-season games)
    current_season_games = games[(games.SEASON_ID == '22023') & (games.GAME_DATE >= '2023-10-24')]
    games_per_team.extend(current_season_games.itertuples(index=False))

# Convert the list of games to a DataFrame
games_per_team_df = pd.DataFrame(games_per_team, columns=current_season_games.columns)

# Initialize the list for target stats
target_stats = []

# Iterate over each team in the list of games
for team_abbr in games_per_team_df['TEAM_ABBREVIATION'].unique():
    team_games = games_per_team_df[games_per_team_df['TEAM_ABBREVIATION'] == team_abbr]
    
    # Take number of Wins and Losses
    WL = team_games.WL.value_counts().reset_index()
    W = WL[WL['WL'] == 'W']['count'].values[0] if 'W' in WL['WL'].values else 0
    L = WL[WL['WL'] == 'L']['count'].values[0] if 'L' in WL['WL'].values else 0
    
    # Initialize empty lists for target stats
    offensive_rating_list = []
    defensive_rating_list = []
    possessions_list = []
    
    # Determine if the team is the away or home team and add stats
    for game_id in team_games.GAME_ID.tolist():
        stats = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=game_id)
        game_row = team_games[team_games.GAME_ID == game_id]
        
        if game_row.MATCHUP.values[0][4] == "@":  # Away Team
            offensive_rating_list.extend([stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['offensiveRating']])
            defensive_rating_list.extend([stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['defensiveRating']])
            possessions_list.extend([stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['possessions']])
        else:  # Home Team
            offensive_rating_list.extend([stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['offensiveRating']])
            defensive_rating_list.extend([stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['defensiveRating']])
            possessions_list.extend([stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['possessions']])
    
    # Combine the lists into a dictionary
    team_stats = {
        'team_abbreviation': team_abbr,
        'team_name': team_games.TEAM_NAME.unique()[0],
        'game_date': team_games.GAME_DATE.tolist(),
        'W': W,
        'L': L,
        'offensive_rating': offensive_rating_list, 
        'defensive_rating': defensive_rating_list,
        'possessions': possessions_list
    }

# Add the team stats dictionary to the target_stats list
target_stats = target_stats + [team_stats]

# Convert the list of target stats to a DataFrame
data = pd.DataFrame(target_stats)

# Compute metrics
data['GP'] = data.W + data.L
data['PCT_W'] = (data.W / data.GP) * 100
data['PCT_W_rank'] = data.PCT_W.rank(method='min', ascending=False).astype('int64')

In [ ]:
data

In [ ]:
### Data preparation for the dashboard
df = data
df.offensive_rating = df.offensive_rating.apply(np.array)
df.defensive_rating = df.defensive_rating.apply(np.array)

df.offensive_rating = df.offensive_rating.apply(np.mean).round(1)
df.defensive_rating = df.defensive_rating.apply(np.mean).round(1)

df['net_rating'] = df.offensive_rating - df.defensive_rating
df.net_rating = df.net_rating.round(1)
df['label'] = df.PCT_W_rank.astype('str') + '-' + df.team_abbreviation.astype('str')
df.sort_values(by='PCT_W_rank', inplace=True)

### Team colors for scatter markers
colors = pd.DataFrame({
    'team_abbreviation':['BOS', 'MIN', 'MIL', 'PHI', 'DEN', 'OKC', 'SAC', 'ORL', 'DAL',
       'LAC', 'MIA', 'NYK', 'CLE', 'NOP', 'HOU', 'LAL', 'GSW', 'IND',
       'PHX', 'BKN', 'CHI', 'ATL', 'UTA', 'TOR', 'MEM', 'CHA', 'POR',
       'WAS', 'SAS', 'DET'],
    'team_color':['#008248','#236192','#00471b','#006bb6','#0d2240','#007ac1','#5b2b82','#0b77bd','#007dc5',
                  '#1d428a','#98002e','#f58426','#6f2633','#b4975a','#ce1141','#552583','#fdb927','#002d62',
                  '#b95915','Black','#ce1141','#e03a3e','#2b5134','#a0a0a3','#5d76a9','#00788c','#cf0a2c',
                  '#002b5c','Black','#1d428a'
                  ]
})

df = df.merge(colors, on='team_abbreviation', how='left')

### Plotly Configuration

### Scatter plot
scatter = go.Scatter(
    x=df['offensive_rating'],
    y=df['defensive_rating'],
    name='team',
    mode='markers+text',
    marker=dict(color=df['team_color'], size=7),
    #marker_symbol='diamond-wide',
    text=df['label'],
    textfont={'color':df['team_color'], 'size':7},
    textposition='top center',
    texttemplate='<b>[%{text}]</b>',
    hovertemplate = '%{text} <br>off_rtg: %{x} </br> def_rtg: %{y}'
)

### Table plot
table = go.Table(
    header=dict(values=list(['Team','rank','GP','W','L','off_r','def_r','net_r']),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[df['team_abbreviation'],
                       df['PCT_W_rank'],
                       df['GP'],
                       df['W'],
                       df['L'],
                       df['offensive_rating'],
                       df['defensive_rating'],
                       df['net_rating']
                       ],
               fill_color='snow',
               align='center'),
    domain=dict(x=[0.6, 1],
                y=[0, 1])
)

### Merge plots
layout = dict(xaxis1=dict( dict(domain=[0, 0.58], anchor='y1')),
              yaxis1=dict( dict(domain=[0, 1], anchor='x1')),
              margin=dict(l=50, r=30, t=45, b=50),
              title='NBA Teams Advanced Statistics'
             )
fig = go.Figure(data = [scatter,table], layout = layout)

### Configuration
fig.add_shape(type="line",
    x0=105, y0=105, x1=125, y1=125,
    line=dict(color="RoyalBlue",width=1)
)
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.update_layout(xaxis_title='Offensive rating', 
                  yaxis_title='Defensive rating',
                  plot_bgcolor='whitesmoke',
                  font_family='Roboto',
                  font=dict(
                    size=11,
                    color='#044575'
                  )
                  )

### Let's plot
iplot(fig, filename = 'basic_table')